In [77]:
import sqlite3
import pandasql as ps

import pandas as pd
import numpy as np

import openpyxl

In [85]:
#read in source tables 
DB_PATH = r'D:FIA\FIADB_CA.db'

con = sqlite3.connect(DB_PATH)
# grab NC stands
stands = pd.read_sql_query("select STAND_CN, STAND_ID, INV_YEAR from FVS_STANDINIT_COND where VARIANT = 'NC'", con)

In [39]:
# scrape stand_id so remeasurement ids match
#stands['INV_YEAR'] = stands['INV_YEAR'].astype(int)
stands['STAND_ID'] = stands['STAND_ID'].str[0:3] + stands['STAND_ID'].str[10:]

In [41]:
# drop stands with no remeasurment data
stands = stands.sort_values(by=['STAND_ID', 'INV_YEAR'])
remeasured_stands = stands[stands.duplicated(subset=['STAND_ID'], keep=False)]

In [63]:
# grab the trees and cond tables
cond = pd.read_sql_query("select CN, PLT_CN, CONDID from COND", con)
trees = pd.read_sql_query("select PLT_CN, CONDID, INVYR, TREE, STATUSCD, SPCD, DIA, HT, VOLCFNET, VOLCFGRS from TREE", con)

In [70]:
# use cond table to get plt_cn to match with trees table
trees_id = trees.merge(cond, how='left', on=['PLT_CN', 'CONDID'])

390955

In [75]:
# grab tress for stands in our remeasured stands database
stand_trees = ps.sqldf("""select PLT_CN, CONDID, TREE, STATUSCD, SPCD, DIA, HT, VOLCFNET, VOLCFGRS, STAND_ID from trees_id t join remeasured_stands s on t.CN = s.STAND_CN AND t.INVYR = s.INV_YEAR""")

In [84]:
# now we need the fia equation for region and species
species = stand_trees['SPCD'].unique()
species.sort()
species

## create matrix for fia equation

array([ 15.,  17.,  20.,  21.,  22.,  41.,  54.,  55.,  81.,  92.,  93.,
        98., 103., 108., 109., 116., 117., 119., 120., 122., 124., 127.,
       202., 211., 231., 242., 251., 263., 264., 312., 313., 333., 351.,
       352., 361., 431., 492., 510., 511., 542., 631., 730., 748., 763.,
       768., 801., 805., 807., 815., 818., 821., 826., 839., 981., 998.])

In [78]:
# export to excel for use in National Volume Estimator Library (NVEL)
stand_trees.to_excel(r'D:\fvs_calibration\data\interim\NC_trees_for_NVEL.xlsx')